In [1]:
import nltk
from nltk import ngrams, word_tokenize, RegexpTokenizer, FreqDist
from pprint import pprint
import numpy as np
import pandas as pd


In [2]:
## books liscensces removed
books = ["SHolmes.txt","Frankenstein.txt","Dracula.txt"]

raws = []
for book in books:
    f = open(book,"r")
    raws.append(f.read())
    f.close()
    
raws = [raw.lower() for raw in raws]

print(len(raws))

tokenizer = RegexpTokenizer(r'[A-Za-z\']+').tokenize

tokensL = [tokenizer(raw) for raw in raws]

3


In [ ]:
n = 3
ngs = [ngrams(tokens, n) for tokens in tokensL]
# ng_hol = ngrams(t_hol, n)
# ng_fra = ngrams(t_fra, n)
# ng_dra = ngrams(t_dra, n)

# ngs = [ng_hol, ng_fra, ng_dra]

In [3]:
lexicon = set().union(*tokensL)
lexicon = sorted(lexicon)

counts = pd.DataFrame(0, lexicon, lexicon)
#pd.DataFrame(0, index=np.arange(1, 4), columns=np.arange(24))

In [ ]:
num_words = len([t for T in tokensL for t in T])
num_words

In [ ]:
len(lexicon)

In [ ]:
num_words / len(lexicon)

In [ ]:
counts.loc["'a","'a"] = 1

In [ ]:
counts.loc["'a","'a"]

In [6]:
def bigramCounts (tokens, lexicon):
    #len_lex = len(lexicon)
    counts = pd.DataFrame(0, lexicon, lexicon)
    for i in range(len(tokens) - 1):
        counts.loc[tokens[i],tokens[i+1]] += 1
        
    return counts

In [ ]:
counts = bigramCounts(tokensL[0],lexicon)

In [ ]:
counts.describe()